In [1]:
import pandas as pd
from utils import *

In [2]:
train = pd.read_csv('/opt/ml/dataset/train/train.csv')
valid = pd.read_csv('/opt/ml/dataset/test/test_data.csv')

In [3]:
helper = DataHelper(data_dir='/opt/ml/dataset/train/entity_train.csv')

for k, (ti, vi) in enumerate(helper.split(ratio=0.2, n_splits=5, mode='skf')):
    train_idxs = ti
    val_idxs = vi
    break



In [4]:
train_data = train[train.columns].iloc[train_idxs] 
valid_data = train[train.columns].iloc[val_idxs] 

In [5]:
len(train_data['sentence'])

25976

In [6]:
len(valid['sentence'])

7765

In [7]:
temp = pd.concat([train_data['sentence'],valid['sentence']])

In [8]:
len(temp) == len(valid['sentence']) + len(train_data['sentence'])

True

In [9]:
temp.to_csv('train.txt', index=False)
valid_data['sentence'].to_csv('valid.txt', index=False)

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, DataCollatorWithPadding, DataCollatorForLanguageModeling,EarlyStoppingCallback
import torch
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments
import wandb

In [2]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForMaskedLM.from_pretrained('klue/roberta-large')
model.to(device)

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=512
)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='valid.txt',
    block_size=512
)

data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)
wandb.login()
wandb.init(
        project="klue",
        entity="chungye-mountain-sherpa",
        name='mlm_roberta',
        group='mlm',
    )


training_args = TrainingArguments(
    output_dir="./kluebert-retrained",
    overwrite_output_dir=True,
    learning_rate=1e-5,
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_steps=250,
    eval_steps=250,
    save_total_limit=2,
    seed=42,
    save_strategy='steps',
    evaluation_strategy='steps',
    gradient_accumulation_steps=4,
    logging_steps=250,
    fp16=True,
    fp16_opt_level='O1',
    report_to="wandb",
    load_best_model_at_end = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

trainer.train()
trainer.save_model("./kluebert-retrained")

/opt/conda/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ddobokki (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Using amp fp16 backend
***** Running training *****
  Num examples = 33742
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 5260
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
250,1.893800,1.549019
500,1.623600,1.478278
750,1.568400,1.452323
1000,1.525000,1.425403
1250,1.499900,1.418164
1500,1.479200,1.391726
1750,1.441100,1.406702
2000,1.434400,1.370924


***** Running Evaluation *****
  Num examples = 6495
  Batch size = 32
Saving model checkpoint to ./kluebert-retrained/checkpoint-250
Configuration saved in ./kluebert-retrained/checkpoint-250/config.json
Model weights saved in ./kluebert-retrained/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6495
  Batch size = 32
Saving model checkpoint to ./kluebert-retrained/checkpoint-500
Configuration saved in ./kluebert-retrained/checkpoint-500/config.json
Model weights saved in ./kluebert-retrained/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6495
  Batch size = 32
Saving model checkpoint to ./kluebert-retrained/checkpoint-750
Configuration saved in ./kluebert-retrained/checkpoint-750/config.json
Model weights saved in ./kluebert-retrained/checkpoint-750/pytorch_model.bin
Deleting older checkpoint [kluebert-retrained/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 6495
  Batch si

RuntimeError: [enforce fail at inline_container.cc:274] . unexpected pos 2492085504 vs 2492085392

In [3]:
# output_dir=args.output_dir,
# per_device_train_batch_size=hp_config['batch_size'],
# per_device_eval_batch_size=hp_config['batch_size'],
# gradient_accumulation_steps=hp_config['gradient_accumulation_steps'],
# learning_rate=hp_config['learning_rate'],
# weight_decay=hp_config['weight_decay'],
# num_train_epochs=hp_config['epochs'],
# logging_dir=args.logging_dir,
# logging_steps=50,
# save_total_limit=2,
# evaluation_strategy=args.eval_strategy,
# eval_steps=50,
# save_steps=50,
# # dataloader_num_workers=4,
# load_best_model_at_end=True,
# fp16=True,
# fp16_opt_level='O1'

In [3]:
trainer.save_model("./kluebert-retrained")

Saving model checkpoint to ./kluebert-retrained
Configuration saved in ./kluebert-retrained/config.json
Model weights saved in ./kluebert-retrained/pytorch_model.bin


In [1]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments

In [2]:
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

In [3]:
for param in model.parameters():
    param.requires_grad = False


for param in model.classifier.parameters():
    param.requires_grad = True

for param in model.roberta.encoder.layer[-1].parameters():
    param.requires_grad = True

In [4]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [15]:
model.roberta.encoder.layer[-1]

RobertaLayer(
  (attention): RobertaAttention(
    (self): RobertaSelfAttention(
      (query): Linear(in_features=1024, out_features=1024, bias=True)
      (key): Linear(in_features=1024, out_features=1024, bias=True)
      (value): Linear(in_features=1024, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): RobertaSelfOutput(
      (dense): Linear(in_features=1024, out_features=1024, bias=True)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): RobertaIntermediate(
    (dense): Linear(in_features=1024, out_features=4096, bias=True)
  )
  (output): RobertaOutput(
    (dense): Linear(in_features=4096, out_features=1024, bias=True)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)